In [1]:
import os
import pandas as pd
import geopandas as gpd

# Display options
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


In [2]:
DATA_PROCESSED = os.path.join("..", "data", "processed")

building_current_2263 = gpd.read_parquet(
    os.path.join(DATA_PROCESSED, "building_current_2263.parquet")
)

building_historic_2263 = gpd.read_parquet(
    os.path.join(DATA_PROCESSED, "building_historic_2263.parquet")
)

print(building_current_2263.shape, building_historic_2263.shape)


(1082999, 18) (49059, 19)


In [3]:
PATH_NTA = os.path.join("..", "data", "raw", "nta", "nyc_nta_2020.zip")

nta_gdf = gpd.read_file(f"zip://{PATH_NTA}")

print(nta_gdf.shape)
nta_gdf.head()


(262, 12)


,borocode,boroname,countyfips,nta2020,ntaname,ntaabbrev,ntatype,cdta2020,cdtaname,shape_leng,shape_area,geometry
0,3.0,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28919.561151,3.532181e+07,"POLYGON ((-73.93213 40.72816, -73.93238 40.727..."
1,3.0,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28134.082661,2.885285e+07,"POLYGON ((-73.95814 40.7244, -73.95772 40.7242..."
2,3.0,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),18250.280091,1.520896e+07,"POLYGON ((-73.95024 40.70547, -73.94984 40.705..."
3,3.0,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),43184.800376,5.226741e+07,"POLYGON ((-73.92406 40.71411, -73.92404 40.714..."
4,3.0,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,14312.192285,9.982023e+06,"POLYGON ((-73.99236 40.68969, -73.99436 40.690..."


In [10]:
nta_2263 = nta_gdf.to_crs("EPSG:2263")

print("NTA CRS after reprojection:", nta_2263.crs)



NTA CRS after reprojection: EPSG:2263


In [7]:
print("NTA columns:")
for c in nta_gdf.columns:
    print(" ", c)

print("\nDtypes:")
print(nta_gdf.dtypes)

nta_gdf.head(3)

print("NTA CRS:", nta_gdf.crs)


NTA columns:
  borocode
  boroname
  countyfips
  nta2020
  ntaname
  ntaabbrev
  ntatype
  cdta2020
  cdtaname
  shape_leng
  shape_area
  geometry

Dtypes:
borocode       float64
boroname        object
countyfips      object
nta2020         object
ntaname         object
ntaabbrev       object
ntatype         object
cdta2020        object
cdtaname        object
shape_leng     float64
shape_area     float64
geometry      geometry
dtype: object
NTA CRS: EPSG:4326


In [8]:
nta_gdf = nta_gdf.rename(columns={
    "nta2020": "NTACode",
    "ntaname": "NTAName"
})

nta_gdf[["NTACode", "NTAName"]].head()


,NTACode,NTAName
0,BK0101,Greenpoint
1,BK0102,Williamsburg
2,BK0103,South Williamsburg
3,BK0104,East Williamsburg
4,BK0201,Brooklyn Heights


In [11]:
building_current_nta = gpd.sjoin(
    building_current_2263,
    nta_2263[["NTACode", "NTAName", "geometry"]],
    how="left",
    predicate="intersects",
)

building_historic_nta = gpd.sjoin(
    building_historic_2263,
    nta_2263[["NTACode", "NTAName", "geometry"]],
    how="left",
    predicate="intersects",
)


In [12]:
current_nta_area = (
    building_current_nta
    .groupby(["NTACode", "NTAName"])["geom_area_sqft"]
    .sum()
    .reset_index()
    .rename(columns={"geom_area_sqft": "current_area_sqft"})
)

historic_nta_area = (
    building_historic_nta
    .groupby(["NTACode", "NTAName"])["geom_area_sqft"]
    .sum()
    .reset_index()
    .rename(columns={"geom_area_sqft": "historic_area_sqft"})
)

nta_growth = current_nta_area.merge(historic_nta_area, on=["NTACode", "NTAName"], how="left")
nta_growth["historic_area_sqft"] = nta_growth["historic_area_sqft"].fillna(0)
nta_growth["growth_sqft"] = nta_growth["current_area_sqft"] - nta_growth["historic_area_sqft"]
nta_growth["growth_pct"] = (nta_growth["growth_sqft"] / nta_growth["historic_area_sqft"].replace(0, pd.NA)) * 100


In [13]:
nta_growth.sort_values("growth_sqft", ascending=False).head(20)


,NTACode,NTAName,current_area_sqft,historic_area_sqft,growth_sqft,growth_pct
248,SI0204,New Springville-Willowbrook-Bulls Head-Travis,2.155542e+07,9.720428e+05,2.058338e+07,2117.538309
251,SI0302,Great Kills-Eltingville,1.974234e+07,7.407551e+05,1.900158e+07,2565.163961
205,QN1001,South Ozone Park,1.936370e+07,5.311895e+05,1.883251e+07,3545.346825
174,QN0501,Maspeth,1.950154e+07,1.391238e+06,1.811030e+07,1301.740555
35,BK1101,Bensonhurst,1.789931e+07,5.089434e+05,1.739037e+07,3416.955042
31,BK1001,Bay Ridge,1.755557e+07,2.536492e+05,1.730192e+07,6821.199682
253,SI0304,Annadale-Huguenot-Prince's Bay-Woodrow,1.814380e+07,1.052359e+06,1.709145e+07,1624.108125
61,BK1803,Canarsie,1.723775e+07,7.634451e+05,1.647431e+07,2157.890403
20,BK0601,Carroll Gardens-Cobble Hill-Gowanus-Red Hook,1.880017e+07,2.996632e+06,1.580354e+07,527.376643
3,BK0104,East Williamsburg,1.745002e+07,1.690818e+06,1.575920e+07,932.046087


In [14]:
nta_growth.sort_values("growth_pct", ascending=False).head(20)


,NTACode,NTAName,current_area_sqft,historic_area_sqft,growth_sqft,growth_pct
257,SI9592,Miller Field,2.308942e+05,354.442511,2.305398e+05,65042.929899
222,QN1304,Cambria Heights,6.472448e+06,34045.296897,6.438403e+06,18911.284958
168,QN0271,Calvary & Mount Zion Cemeteries,3.554360e+05,2186.234453,3.532497e+05,16157.907243
130,MN0601,Stuyvesant Town-Peter Cooper Village,1.146778e+06,7441.864033,1.139336e+06,15309.815874
57,BK1704,East Flatbush-Remsen Village,6.371341e+06,42205.422270,6.329136e+06,14996.024676
13,BK0471,The Evergreens Cemetery,1.017804e+05,775.290035,1.010051e+05,13028.03975
116,BX2891,Pelham Bay Park,2.883385e+05,2302.077713,2.860364e+05,12425.141698
178,QN0571,Mount Olivet & All Faiths Cemeteries,5.642963e+04,532.887433,5.589675e+04,10489.409765
199,QN0891,Cunningham Park,8.996130e+04,850.787407,8.911051e+04,10473.886814
204,QN0905,Woodhaven,7.805141e+06,99692.946058,7.705448e+06,7729.180537


In [15]:
nta_growth.to_parquet(os.path.join(DATA_PROCESSED, "nta_building_growth.parquet"))


In [16]:
pct_current = building_current_nta["NTACode"].notna().mean()
pct_hist = building_historic_nta["NTACode"].notna().mean()

print("Current buildings assigned to NTA:", pct_current)
print("Historic buildings assigned to NTA:", pct_hist)

print("\nMissing NTA in current (sample rows):")
display(building_current_nta.loc[building_current_nta["NTACode"].isna(), ["base_bbl", "bin", "geom_area_sqft"]].head(10))


Current buildings assigned to NTA: 0.9999723037466568
Historic buildings assigned to NTA: 0.9994702742349537

Missing NTA in current (sample rows):


,base_bbl,bin,geom_area_sqft
34497,2059580065,2125133.0,24087.683158
69713,4043460075,4605702.0,42499.337350
97214,4163500002,4559530.0,893.539085
190611,4088100083,4617281.0,238.642943
200875,4141630037,4616016.0,405.645568
265913,4082680103,4613314.0,1076.089052
289292,4155100050,4616233.0,188.152446
391384,4082730019,4613763.0,295.110228
475941,3020230001,3395497.0,3050.707770
502781,4155100048,4516754.0,150.012258


In [17]:
print("Current last_statu counts:")
print(building_current_2263["last_statu"].value_counts(dropna=False).head(20))

print("\nHistoric last_statu counts:")
print(building_historic_2263["last_statu"].value_counts(dropna=False).head(20))


Current last_statu counts:
last_statu
Constructed                 1079925
Alteration                     1736
None                            383
Split                           359
Correction                      133
Merged                          123
Initialization                  109
Marked for Demolition            76
Marked for Construction          66
Investigate Construction         52
Geometry                         17
Demolition                       15
Investigate Demolition            5
Name: count, dtype: int64

Historic last_statu counts:
last_statu
Demolition                  46189
Alteration                   1637
Constructed                   393
Merged                        327
Correction                    267
Split                         155
Demollition                    34
None                           22
Demolitian                     16
Geometry                        7
Marked for Demolition           6
Investigate Demolition          2
Marked for Construct

In [19]:
current_nta_stats = (
    building_current_nta
    .groupby(["NTACode", "NTAName"])
    .agg(
        current_area_sqft=("geom_area_sqft", "sum"),
        current_building_count=("geom_area_sqft", "size"),
    )
    .reset_index()
)

historic_nta_stats = (
    building_historic_nta
    .groupby(["NTACode", "NTAName"])
    .agg(
        historic_area_sqft=("geom_area_sqft", "sum"),
        historic_building_count=("geom_area_sqft", "size"),
    )
    .reset_index()
)

nta_growth = current_nta_stats.merge(historic_nta_stats, on=["NTACode", "NTAName"], how="left")
nta_growth[["historic_area_sqft", "historic_building_count"]] = nta_growth[["historic_area_sqft", "historic_building_count"]].fillna(0)

nta_growth["growth_sqft"] = nta_growth["current_area_sqft"] - nta_growth["historic_area_sqft"]
nta_growth["growth_pct"] = (nta_growth["growth_sqft"] / nta_growth["historic_area_sqft"].replace(0, pd.NA)) * 100

nta_growth["building_count_delta"] = nta_growth["current_building_count"] - nta_growth["historic_building_count"]


nta_growth.sort_values("growth_sqft", ascending=False).head(20)



,NTACode,NTAName,current_area_sqft,current_building_count,historic_area_sqft,historic_building_count,growth_sqft,growth_pct,building_count_delta
248,SI0204,New Springville-Willowbrook-Bulls Head-Travis,2.155542e+07,11285,9.720428e+05,350.0,2.058338e+07,2117.538309,10935.0
251,SI0302,Great Kills-Eltingville,1.974234e+07,17630,7.407551e+05,640.0,1.900158e+07,2565.163961,16990.0
205,QN1001,South Ozone Park,1.936370e+07,23688,5.311895e+05,528.0,1.883251e+07,3545.346825,23160.0
174,QN0501,Maspeth,1.950154e+07,11584,1.391238e+06,373.0,1.811030e+07,1301.740555,11211.0
35,BK1101,Bensonhurst,1.789931e+07,15186,5.089434e+05,448.0,1.739037e+07,3416.955042,14738.0
31,BK1001,Bay Ridge,1.755557e+07,13882,2.536492e+05,203.0,1.730192e+07,6821.199682,13679.0
253,SI0304,Annadale-Huguenot-Prince's Bay-Woodrow,1.814380e+07,12519,1.052359e+06,843.0,1.709145e+07,1624.108125,11676.0
61,BK1803,Canarsie,1.723775e+07,14365,7.634451e+05,290.0,1.647431e+07,2157.890403,14075.0
20,BK0601,Carroll Gardens-Cobble Hill-Gowanus-Red Hook,1.880017e+07,7462,2.996632e+06,553.0,1.580354e+07,527.376643,6909.0
3,BK0104,East Williamsburg,1.745002e+07,4939,1.690818e+06,637.0,1.575920e+07,932.046087,4302.0


In [20]:
# Bring in NTA land area in sqft from NTA geometry
nta_land = nta_2263.copy()
nta_land["nta_land_area_sqft"] = nta_land.geometry.area
nta_land = nta_land[["NTACode", "nta_land_area_sqft"]]

nta_growth = nta_growth.merge(nta_land, on="NTACode", how="left")

nta_growth["current_built_area_ratio"] = nta_growth["current_area_sqft"] / nta_growth["nta_land_area_sqft"]
nta_growth["historic_built_area_ratio"] = nta_growth["historic_area_sqft"] / nta_growth["nta_land_area_sqft"]
nta_growth["built_area_ratio_delta"] = nta_growth["current_built_area_ratio"] - nta_growth["historic_built_area_ratio"]
